In [54]:
import boto3
import numpy as np
import pandas as pd

In [19]:
# get list of files in s3

s3 = boto3.resource('s3')
my_bucket = s3.Bucket('himi-car-classification')

#### Make list of keys

In [55]:
train_lst = []
for object_summary in my_bucket.objects.filter(Prefix="images/train/"):
    train_lst.append(str(object_summary.key.split('/')[-2]) + '/' + str(object_summary.key.split('/')[-1]))
    
unique_cars = np.array([x.split('/')[0] for x in train_lst])
car_list = np.unique(unique_cars)
car_list_dict = {}
iterator = 0
for car in car_list:
    car_list_dict.setdefault(car, iterator)
    iterator += 1

#### Save as .lst format
Requirements: https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html#IC-inputoutput

In [59]:
train_df = pd.DataFrame()
idx=0
for c in train_lst[1:]:
    train_df = train_df.append({"index": idx, 'class': car_list_dict[c.split('/')[0]], 'key': c}, ignore_index=True)
    idx+=1

In [62]:
train_df['class'] = train_df['class'].astype(int)
train_df['index'] = train_df['index'].astype(int)
train_df = train_df[['index', 'class', 'key']]

train_df.to_csv('train_lst.lst', sep='\t', header=False, index=False)

In [78]:
len(train_lst)

39782

In [71]:
test_lst = []
for object_summary in my_bucket.objects.filter(Prefix="images/test/"):
    test_lst.append(str(object_summary.key.split('/')[-2]) + '/' + str(object_summary.key.split('/')[-1]))

In [72]:
test_df = pd.DataFrame()
idx=0
for c in test_lst[1:]:
    test_df = test_df.append({"index": idx, 'class': car_list_dict[c.split('/')[0]], 'key': c}, ignore_index=True)
    idx+=1

In [73]:
test_df['class'] = test_df['class'].astype(int)
test_df['index'] = test_df['index'].astype(int)
test_df = test_df[['index', 'class', 'key']]

test_df.to_csv('validation_lst.lst', sep='\t', header=False, index=False)

In [12]:
with open('train.lst', 'wb') as f:
    f.write(BytesIO(train_lst))

TypeError: a bytes-like object is required, not 'list'